# comfirm interpolation
### 3D scalar in taichi

In [10]:
import numpy as np
import taichi as ti
import taichi_glsl as ts
ti.init(arch = ti.cpu)

shape = (5, 5, 6)
field =  ti.field(ti.f64, shape = shape)
np.random.seed(3)
self = np.random.rand(shape[0], shape[1], shape[2])
field.from_numpy(self)
P = ti.Vector([1,1.5,2.005])



@ti.func
def trilerp(field: ti.template(), P):
    '''
    Tilinear sampling an 3D field with a real index.
    :parameter field: (3D Tensor)
        Specify the field to sample.
    :parameter P: (3D Vector of float)
        Specify the index in field.
    :note:
        If one of the element to be accessed is out of `field.shape`, then
        `Tilerp` will automatically do a clamp for you, see :func:`sample`.
        Syntax ref : https://en.wikipedia.org/wiki/Trilinear_interpolation.
    :return:
        The return value is calcuated as::
            I = int(P)
            w0 = ts.fract(P)
            w1 = 1.0 - w0
            c00 = ts.sample(field, I + ts.D.yyy) * w1.x + ts.sample(field, I + ts.D.xyy) * w0.x
            c01 = ts.sample(field, I + ts.D.yyx) * w1.x + ts.sample(field, I + ts.D.xyx) * w0.x
            c10 = ts.sample(field, I + ts.D.yxy) * w1.x + ts.sample(field, I + ts.D.xxy) * w0.x
            c11 = ts.sample(field, I + ts.D.yxx) * w1.x + ts.sample(field, I + ts.D.xxx) * w0.x
            c0 = c00 * w1.y + c10 * w0.y
            c1 = c01 * w1.y + c11 * w0.y
            return c0 * w1.z + c1 * w0.z
        .. where D = vec(1, 0, -1)
    '''
    I = int(P)
    w0 = ts.fract(P)
    w1 = 1.0 - w0

    c00 = ts.sample(field, I + ts.D.yyy) * w1.x + ts.sample(
        field, I + ts.D.xyy) * w0.x
    c01 = ts.sample(field, I + ts.D.yyx) * w1.x + ts.sample(
        field, I + ts.D.xyx) * w0.x
    c10 = ts.sample(field, I + ts.D.yxy) * w1.x + ts.sample(
        field, I + ts.D.xxy) * w0.x
    c11 = ts.sample(field, I + ts.D.yxx) * w1.x + ts.sample(
        field, I + ts.D.xxx) * w0.x

    c0 = c00 * w1.y + c10 * w0.y
    c1 = c01 * w1.y + c11 * w0.y

    return c0 * w1.z + c1 * w0.z


@ti.kernel
def main() -> ti.f64:
    # x =  ts.sampling.sample(field, P)
    x = trilerp(field, P)

    return x
# val = main()
# print(val)
main()

[Taichi] Starting on arch=x64


0.41029143957122105

### in numpy

In [1]:
import numpy as np
from scipy.interpolate import interpn

def field_point(self, coord): # It is normalized. To get original field: u.field_point()*u.magitude_point()
    points = (np.linspace(0,self.shape[0]-1,self.shape[0]), 
              np.linspace(0,self.shape[1]-1,self.shape[1]),
              np.linspace(0,self.shape[2]-1,self.shape[2])
            )
    fx = interpn(points, self, coord)[0]
    # fy = interpn(points, self.field_y, coord)[0]
    # fz = interpn(points, self.field_z, coord)[0]
    # return np.array([fx, fy, fz])
    return fx

shape = (5, 5, 6)
np.random.seed(3)
self = np.random.rand(shape[0], shape[1], shape[2])
print(field_point(self, (1, 1.5,2)))
# print(self[1,1,1])

0.40962558119277803


## 3D vector comfirmation

Since taichi doesn't support to return a list or tuple, I can't return a struct data. The main idea should be ***update*** an initialized output textures. 

In [3]:
import taichi as ti
import taichi_glsl as ts
import numpy as np
ti.init(arch = ti.cpu)

shape = (5, 5, 6)
field =  ti.Vector.field(n = 3, dtype = ti.f64, shape = shape)
np.random.seed(3)
self = np.random.rand(shape[0], shape[1], shape[2])
temp = np.stack((self, self, self), axis = -1)
print(temp.shape)
field.from_numpy(temp)
P = ti.Vector([1,1.5,2])



@ti.func
def trilerp(field: ti.template(), P):
    I = int(P)
    w0 = ts.fract(P)
    w1 = 1.0 - w0

    c00 = ts.sample(field, I + ts.D.yyy) * w1.x + ts.sample(
        field, I + ts.D.xyy) * w0.x
    c01 = ts.sample(field, I + ts.D.yyx) * w1.x + ts.sample(
        field, I + ts.D.xyx) * w0.x
    c10 = ts.sample(field, I + ts.D.yxy) * w1.x + ts.sample(
        field, I + ts.D.xxy) * w0.x
    c11 = ts.sample(field, I + ts.D.yxx) * w1.x + ts.sample(
        field, I + ts.D.xxx) * w0.x

    c0 = c00 * w1.y + c10 * w0.y
    c1 = c01 * w1.y + c11 * w0.y

    return c0 * w1.z + c1 * w0.z


@ti.kernel
def main() -> ti.f64:
    # x =  ts.sampling.sample(field, P)
    x = trilerp(field, P)
    # return x # is not supported in taichi currently
    return x[1]
val = main()
print(val)
# main()

[Taichi] Starting on arch=x64
(5, 5, 6, 3)
0.40962558119277803


In [ ]:
# from extrnal array

import taichi as ti
import numpy as np

ti.init()

n = 4
m = 7

val = ti.field(ti.i32, shape=(n, m))

@ti.kernel
def test_numpy(arr: ti.ext_arr()):
  for i in range(n):
    for j in range(m):
      arr[i, j] += i + j

a = np.empty(shape=(n, m), dtype=np.int32)

for i in range(n):
  for j in range(m):
    a[i, j] = i * j

test_numpy(a)

for i in range(n):
  for j in range(m):
    assert a[i, j] == i * j + i + j

[Taichi] Starting on arch=cuda


TypeError: 'list' object cannot be interpreted as an integer

In [30]:
from scipy import interpolate
from scipy.ndimage import map_coordinates

z = np.arange(25).reshape(5, 5)
print(z)
f = interpolate.interp2d(np.arange(0,50,10), np.arange(0,50,10), z)
f(np.arange(0,50,5), np.arange(0,50,5))

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]


array([[ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4. ],
       [ 2.5,  3. ,  3.5,  4. ,  4.5,  5. ,  5.5,  6. ,  6.5,  6.5],
       [ 5. ,  5.5,  6. ,  6.5,  7. ,  7.5,  8. ,  8.5,  9. ,  9. ],
       [ 7.5,  8. ,  8.5,  9. ,  9.5, 10. , 10.5, 11. , 11.5, 11.5],
       [10. , 10.5, 11. , 11.5, 12. , 12.5, 13. , 13.5, 14. , 14. ],
       [12.5, 13. , 13.5, 14. , 14.5, 15. , 15.5, 16. , 16.5, 16.5],
       [15. , 15.5, 16. , 16.5, 17. , 17.5, 18. , 18.5, 19. , 19. ],
       [17.5, 18. , 18.5, 19. , 19.5, 20. , 20.5, 21. , 21.5, 21.5],
       [20. , 20.5, 21. , 21.5, 22. , 22.5, 23. , 23.5, 24. , 24. ],
       [20. , 20.5, 21. , 21.5, 22. , 22.5, 23. , 23.5, 24. , 24. ]])

In [61]:
temp = np.arange(10).reshape(2,5)
def bilinear_interpolation(data_in, resample_factor):
        from scipy import interpolate
        x_grid = np.linspace(0, data_in.shape[1]-1, data_in.shape[1])
        y_grid = np.linspace(0, data_in.shape[0]-1, data_in.shape[0])
        f = interpolate.interp2d(x_grid, y_grid, data_in, kind='linear')
        return f(np.linspace(0, data_in.shape[1], data_in.shape[1] * resample_factor), 
                    np.linspace(0, data_in.shape[0], data_in.shape[0] * resample_factor))
        
        
print(temp)
bilinear_interpolation(temp, 2)

[[0 1 2 3 4]
 [5 6 7 8 9]]


array([[0.        , 0.55555556, 1.11111111, 1.66666667, 2.22222222,
        2.77777778, 3.33333333, 3.88888889, 4.        , 4.        ],
       [3.33333333, 3.88888889, 4.44444444, 5.        , 5.55555556,
        6.11111111, 6.66666667, 7.22222222, 7.33333333, 7.33333333],
       [5.        , 5.55555556, 6.11111111, 6.66666667, 7.22222222,
        7.77777778, 8.33333333, 8.88888889, 9.        , 9.        ],
       [5.        , 5.55555556, 6.11111111, 6.66666667, 7.22222222,
        7.77777778, 8.33333333, 8.88888889, 9.        , 9.        ]])

In [1]:
# generate video from pngs
import cv2
import numpy as np
import os


def generate_video(image_folder, video_name, fps=30):
    images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(video_name, 0, fps, (width,height))

    for image in images:
        # print("Processing frame: " + image)
        video.write(cv2.imread(os.path.join(image_folder, image)))

    cv2.destroyAllWindows()
    video.release()

generate_video('D:/CUHK/SbCodes/LIC/animated_lic/images/', 'test.mp4')

Processing frame: image_00000.png
Processing frame: image_00001.png
Processing frame: image_00002.png
Processing frame: image_00003.png
Processing frame: image_00004.png
Processing frame: image_00005.png
Processing frame: image_00006.png
Processing frame: image_00007.png
Processing frame: image_00008.png
Processing frame: image_00009.png
Processing frame: image_00010.png
Processing frame: image_00011.png
Processing frame: image_00012.png
Processing frame: image_00013.png
Processing frame: image_00014.png
Processing frame: image_00015.png
Processing frame: image_00016.png
Processing frame: image_00017.png
Processing frame: image_00018.png
Processing frame: image_00019.png
Processing frame: image_00020.png
Processing frame: image_00021.png
Processing frame: image_00022.png
Processing frame: image_00023.png
Processing frame: image_00024.png
Processing frame: image_00025.png
Processing frame: image_00026.png
Processing frame: image_00027.png
Processing frame: image_00028.png
Processing fra